In [37]:
import pandas as pd

In [ ]:
df_target = pd.read_csv('data/02_input_target.csv')
df_capacity = pd.read_csv('data/02_input_capacity.csv')

In [24]:
((A['Quantity'] - B['Quantity'])**2).sum()/len(A)

np.float64(38.45)

In [39]:
df_capacity

,Country,Monthly Capacity
0,Australia,105000
1,Brazil,90000
2,Egypt,90000
3,France,30000
4,India,90000
5,Italy,85000
6,Japan,45000
7,Russia,105000
8,South Africa,75000
9,United Kingdom,55000


In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp

# 1. Lettura dataset (puoi modificare con i tuoi file)
demand_df = df_target        # Colonne: Country, Product, Month, Quantity
capacity_df = df_capacity     # Colonne: Country, Monthly Capacity

# 2. Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

plant_capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))
plants = list(plant_capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# 3. Inizializza solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# 4. Variabili decisionali: quanto ogni impianto i produce per domanda (j, p, m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# 5. Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 6. Vincoli: soddisfare domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 7. Vincoli: capacità mensile di ogni impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= plant_capacity[i])
        # Calcolo del carico relativo
        solver.Add(monthly_prod <= max_load * plant_capacity[i])

# 8. Obiettivo: minimizzare il carico massimo
solver.Minimize(max_load)

# 9. Risoluzione
status = solver.Solve()

# 10. Output
if status == pywraplp.Solver.OPTIMAL:
    print(f'Status: Ottimale\nCarico massimo usato: {max_load.solution_value():.2%}')
    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            print(f'{i} produce {var.solution_value():.0f} unità per {j} ({p}) a {m}')
else:
    print('Nessuna soluzione ottimale trovata.')


Status: Ottimale
Carico massimo usato: 90.23%
Australia produce 1649 unità per Japan (FreshStart Toothpaste) a Apr2004
Australia produce 0 unità per Japan (FreshStart Toothpaste) a May2004
Australia produce 0 unità per Japan (FreshStart Toothpaste) a Sep2004
Australia produce 1567 unità per Japan (FreshStart Toothpaste) a Oct2004
Australia produce 0 unità per Japan (FreshStart Toothpaste) a Dec2004
Australia produce 684 unità per Japan (SmileShield Whitening Strips) a Mar2004
Australia produce 0 unità per Japan (SmileShield Whitening Strips) a Jul2004
Australia produce 1406 unità per Japan (LuminousLip Care) a Jan2004
Australia produce 1396 unità per Japan (LuminousLip Care) a Sep2004
Australia produce 0 unità per Japan (LuminousLip Care) a Nov2004
Australia produce 0 unità per Japan (CleanSlate Mouthwash) a Jan2004
Australia produce 506 unità per Japan (CleanSlate Mouthwash) a Apr2004
Australia produce 0 unità per Japan (CleanSlate Mouthwash) a Jul2004
Australia produce 527 unità per 

In [43]:
# initialize two dataframes for the solution
# production_plan = pd.DataFrame(columns=['Country', 'Product', 'Month', 'Quantity'])
# shipment_plan = pd.DataFrame(columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity'])

shipment_data = []

# fill the production_plan dataframe with the results
for (i, j, p, m), var in x.items():
	shipment_data.append({
        'Origin': i,
		'Destination': j,
		'Product': p,
		'Month': m,
		'Quantity': int(var.solution_value())
	})
    
shipment_plan = pd.DataFrame(shipment_data)

In [44]:
shipment_plan[(~(shipment_plan['Origin'] == shipment_plan['Destination'])) & (shipment_plan['Quantity'] > 0)].reset_index(drop=True)

,Origin,Destination,Product,Month,Quantity
0,Australia,Japan,FreshStart Toothpaste,Apr2004,1649
1,Australia,Japan,FreshStart Toothpaste,Oct2004,1567
2,Australia,Japan,SmileShield Whitening Strips,Mar2004,684
3,Australia,Japan,LuminousLip Care,Jan2004,1406
4,Australia,Japan,LuminousLip Care,Sep2004,1396
...,...,...,...,...,...
6804,United Kingdom,Russia,SoothingPaws Cat Lotion,Jun2004,1007
6805,United Kingdom,Russia,BrightBreeze Pet Odor Eliminator,Dec2004,835
6806,United Kingdom,Russia,EasyWash Pet Laundry Detergent,Apr2004,426
6807,United Kingdom,Russia,EasyWash Pet Laundry Detergent,Jun2004,385


In [45]:
shipment_plan.to_csv('outputs/02_output_shipments_1239.csv', index=False)

In [46]:
production_plan = shipment_plan.groupby(['Origin', 'Product', 'Month'], as_index=False).agg({'Quantity': 'sum'})
production_plan.rename(columns={'Origin': 'Country'}, inplace=True)
production_plan.to_csv('outputs/02_output_productionPlan_1239.csv', index=False)

In [41]:
A = df_target.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()
B = shipment_plan.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()

In [42]:
A['Quantity'] - B['Quantity']

0      3
1      8
2      5
3      9
4      4
      ..
95    10
96     5
97     8
98     6
99     7
Name: Quantity, Length: 100, dtype: int64

# Modello buono!!!

In [47]:
import pandas as pd
from ortools.linear_solver import pywraplp

# 1. Lettura dataset (puoi modificare con i tuoi file)
demand_df = df_target        # Colonne: Country, Product, Month, Quantity
capacity_df = df_capacity     # Colonne: Country, Monthly Capacity

# 2. Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

plant_capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))
plants = list(plant_capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# 3. Inizializza solver
solver = pywraplp.Solver.CreateSolver('SAT')

# 4. Variabili decisionali: quanto ogni impianto i produce per domanda (j, p, m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# 5. Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 6. Vincoli: soddisfare domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 7. Vincoli: capacità mensile di ogni impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= plant_capacity[i])
        # Calcolo del carico relativo
        solver.Add(monthly_prod <= max_load * plant_capacity[i])

# 8. Obiettivo: minimizzare il carico massimo
solver.Minimize(max_load)

# 9. Risoluzione
status = solver.Solve()

# 10. Output
if status == pywraplp.Solver.OPTIMAL:
    print(f'Status: Ottimale\nCarico massimo usato: {max_load.solution_value():.2%}')
    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            print(f'{i} produce {var.solution_value():.0f} unità per {j} ({p}) a {m}')
else:
    print('Nessuna soluzione ottimale trovata.')


Status: Ottimale
Carico massimo usato: 100.00%
Australia produce 1649 unità per Japan (FreshStart Toothpaste) a Apr2004
Australia produce 1664 unità per Japan (FreshStart Toothpaste) a May2004
Australia produce 635 unità per Japan (SmileShield Whitening Strips) a May2004
Australia produce 634 unità per Japan (SmileShield Whitening Strips) a Jun2004
Australia produce 682 unità per Japan (SmileShield Whitening Strips) a Oct2004
Australia produce 619 unità per Japan (SmileShield Whitening Strips) a Nov2004
Australia produce 1436 unità per Japan (LuminousLip Care) a Jul2004
Australia produce 538 unità per Japan (CleanSlate Mouthwash) a Nov2004
Australia produce 2148 unità per Japan (DentalDefense Toothpaste) a Dec2004
Australia produce 713 unità per Japan (FreshFusion Mouth Rinse) a Feb2004
Australia produce 728 unità per Japan (FreshFusion Mouth Rinse) a Jun2004
Australia produce 1711 unità per Japan (DailyDazzle Toothpaste) a Jul2004
Australia produce 1614 unità per Japan (DailyDazzle To

In [48]:
# initialize two dataframes for the solution
# production_plan = pd.DataFrame(columns=['Country', 'Product', 'Month', 'Quantity'])
# shipment_plan = pd.DataFrame(columns=['Origin', 'Destination', 'Product', 'Month', 'Quantity'])

shipment_data = []

# fill the production_plan dataframe with the results
for (i, j, p, m), var in x.items():
	shipment_data.append({
        'Origin': i,
		'Destination': j,
		'Product': p,
		'Month': m,
		'Quantity': int(var.solution_value())
	})
    
shipment_plan = pd.DataFrame(shipment_data)

In [49]:
shipment_plan[(~(shipment_plan['Origin'] == shipment_plan['Destination'])) & (shipment_plan['Quantity'] > 0)].reset_index(drop=True)

,Origin,Destination,Product,Month,Quantity
0,Australia,Japan,FreshStart Toothpaste,Apr2004,1649
1,Australia,Japan,FreshStart Toothpaste,May2004,1664
2,Australia,Japan,SmileShield Whitening Strips,May2004,635
3,Australia,Japan,SmileShield Whitening Strips,Jun2004,634
4,Australia,Japan,SmileShield Whitening Strips,Oct2004,682
...,...,...,...,...,...
6631,United Kingdom,Russia,FreshFusion All-Purpose Cleaner,Oct2004,1895
6632,United Kingdom,Russia,GentleGlow Dish Soap,Apr2004,598
6633,United Kingdom,Russia,SoothingPaws Cat Lotion,Jun2004,1007
6634,United Kingdom,Russia,SoothingPaws Cat Lotion,Aug2004,1061


In [50]:
A = df_target.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()
B = shipment_plan.groupby(['Product']).agg({'Quantity': 'sum'}).reset_index()
A['Quantity'] - B['Quantity']

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Name: Quantity, Length: 100, dtype: int64

In [51]:
shipment_plan.to_csv('outputs/02_output_shipments_1239_true.csv', index=False)
production_plan = shipment_plan.groupby(['Origin', 'Product', 'Month'], as_index=False).agg({'Quantity': 'sum'})
production_plan.rename(columns={'Origin': 'Country'}, inplace=True)
production_plan.to_csv('outputs/02_output_productionPlan_1239_true.csv', index=False)